In [ ]:
%cd /content/
! git clone https://github.com/joeynmt/joeynmt.git

In [ ]:
% cd joeynmt/
!pip3 install .

In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
#preprocessing the data into subword tokens
%cd /content/drive/MyDrive/data_en_ro/dcep_training_files

#learn bpe on the training data
#! cat vocab.$src
! subword-nmt learn-joint-bpe-and-vocab --input train.en train.ro -s 30000 -o bpe.codes.30000 --write-vocabulary vocab.en vocab.ro

#apply BPE splits to the dev and test data
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < train.en > train.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < train.ro > train.bpe.ro

! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < val.en > val.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < val.ro > val.bpe.ro

! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < test.en > test.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < test.ro > test.bpe.ro

/content/drive/MyDrive/data_en_ro/dcep_training_files


In [ ]:
! pip install -U sacremoses

! sacremoses -l "en" -j 8 tokenize < train.en > tok_train.en
! sacremoses -l "ro" -j 8 tokenize < train.ro > tok_train.ro

! sacremoses -l "en" -j 8 tokenize < val.en > tok_val.en
! sacremoses -l "ro" -j 8 tokenize < val.ro > tok_val.ro

! sacremoses -l "en" -j 8 tokenize < test.en > tok_test.en
! sacremoses -l "ro" -j 8 tokenize < test.ro > tok_test.ro


Requirement already up-to-date: sacremoses in /usr/local/lib/python3.7/dist-packages (0.0.45)
100% 783896/783896 [02:43<00:00, 4805.20it/s]
100% 783896/783896 [02:04<00:00, 6292.90it/s]
100% 2000/2000 [00:03<00:00, 608.97it/s]
100% 2000/2000 [00:02<00:00, 768.00it/s]
100% 2000/2000 [00:03<00:00, 617.02it/s]
100% 2000/2000 [00:02<00:00, 844.84it/s]


In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_tokenization/

! subword-nmt learn-joint-bpe-and-vocab --input tok_train.en tok_train.ro -s 30000 -o bpe.codes.30000 --write-vocabulary vocab.en vocab.ro

#apply BPE splits to the dev and test data
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < tok_train.en > tok_train.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < tok_train.ro > tok_train.bpe.ro

! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < tok_val.en > tok_val.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < tok_val.ro > tok_val.bpe.ro

! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < tok_test.en > tok_test.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < tok_test.ro > tok_test.bpe.ro

/content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_tokenization


In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE/
!wc -l *

/content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE
    30001 bpe.codes.30000
     2000 test.bpe.en
     2000 test.bpe.ro
   783896 train.bpe.en
   783896 train.bpe.ro
     2000 val.bpe.en
     2000 val.bpe.ro
    21712 vocab.en
    30206 vocab.ro
  1657711 total


In [ ]:
!head /content/drive/MyDrive/data_en_ro/dcep_full/clean_vocab.txt

,
&quot;
the
de
of
a
and
.
to
în


In [ ]:
!wc -l /content/drive/MyDrive/data_en_ro/dcep_full/*.bpe.*

    10000 /content/drive/MyDrive/data_en_ro/dcep_full/clean_dev.bpe.en
    10000 /content/drive/MyDrive/data_en_ro/dcep_full/clean_dev.bpe.ro
   157580 /content/drive/MyDrive/data_en_ro/dcep_full/clean_test.bpe.en
   157580 /content/drive/MyDrive/data_en_ro/dcep_full/clean_test.bpe.ro
   620316 /content/drive/MyDrive/data_en_ro/dcep_full/clean_train.bpe.en
   620316 /content/drive/MyDrive/data_en_ro/dcep_full/clean_train.bpe.ro
  1575792 total


In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE

/content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE


In [ ]:
! sudo chmod 777 /content/joeynmt/scripts/build_vocab.py
! /content/joeynmt/scripts/build_vocab.py train.bpe.en train.bpe.ro --output_path vocab.txt


#some output
! echo "BPE Sentences english"
! head -n 5 /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE/test.bpe.en

! echo "BPE Sentences ro"
! head -n 5 /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE/test.bpe.ro

! echo "Combined BPE Vocab"
! head -n 10 /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE/vocab.txt

BPE Sentences english
Considers the Agency to be a source of important environmental information for all EU institutions and policy-@@ mak@@ ing; notes with satisfaction that the Agency has been able to trans@@ form some complic@@ ated data into clear conclusions and to communicate them to the public, such as the Agency's environmental statement 2006 and specific reports on bi@@ o energy and greenhouse gas emission trends and pro@@ jec@@ tions in Europe;
00@@ 02@@ /2009 )
Urges all the institutions to cooperate construc@@ tively with the Ombudsman at all stages of the procedure, to ab@@ ide by am@@ ic@@ able sett@@ lement@@ s, to follow up his critical remarks and to apply his draft recommend@@ ations;
having regard to its resolution of 14 December 2006 on A European strategy for sustainable, competitive and secure energy − Green paper Texts adopted, P6_TA(2006)0@@ 60@@ 3 .
These pap@@ ers began to be submitted to ACP parliaments at the end of 2007 and it is important for the Committee

In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_full/

/content/drive/MyDrive/data_en_ro/dcep_full


In [ ]:
!wc -l /home/bernadeta/BA_code/data/00-raw/EN-RO-bisentences.txt

25055 clean_vocab.txt
